In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import sys
sys.path.append('/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet')

In [ ]:
import json
import torch
import datetime
import argparse
import numpy as np
from scripts.utils import *
from scripts.model.sqlnet import SQLNet

In [ ]:
dataset = '/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider'
emb_path = '/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/glove.42B.300d.txt'
TOY = False
train_emb = False
N_word = 300
B_word = 42
if TOY:
    USE_SMALL = True
    GPU = False
    BATCH_SIZE = 20
else:
    USE_SMALL = False
    GPU = True
    BATCH_SIZE = 64
TRAIN_ENTRY = (True, True, True)  # (AGG, SEL, COND)
TRAIN_AGG, TRAIN_SEL, TRAIN_COND = TRAIN_ENTRY
learning_rate = 1e-3

In [ ]:
import json
import torch
import datetime
import argparse
import numpy as np
from scripts.utils import *
from scripts.model.sqlnet import SQLNet

In [ ]:
dataset = '/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider'
emb_path = '/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/glove.42B.300d.txt'
TOY = False
train_emb = False
N_word = 300
B_word = 42
if TOY:
    USE_SMALL = True
    GPU = False
    BATCH_SIZE = 20
else:
    USE_SMALL = False
    GPU = True
    BATCH_SIZE = 64
TRAIN_ENTRY = (True, True, True)  # (AGG, SEL, COND)
TRAIN_AGG, TRAIN_SEL, TRAIN_COND = TRAIN_ENTRY
learning_rate = 1e-3

In [ ]:
sql_data, table_data, val_sql_data, val_table_data, test_sql_data, test_table_data, schemas, TRAIN_DB, DEV_DB, TEST_DB = load_dataset(dataset, use_small=USE_SMALL)

word_emb = load_word_emb(emb_path, load_used=train_emb, use_small=USE_SMALL)

model = SQLNet(word_emb, N_word=N_word, gpu=GPU, trainable_emb=train_emb)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=0)
init_acc = epoch_acc(model, BATCH_SIZE, val_sql_data, val_table_data, schemas, TRAIN_ENTRY)

Loading from datasets...
Loading data from /content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider/tables.json
Loading data from /content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider/train_spider.json
Loading data from /content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider/dev.json
Loading data from /content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider/dev.json
Loading word embedding from /content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/glove.42B.300d.txt
Using fixed embedding for words but trainable embedding for types


/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_prob_qc_num = self.softmax(att_val_qc_num.view((-1, max_q_len))).view(B, -1, max_q_len)
/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_prob_qc = self.softmax(att_val_qc.view((-1, max_q_len))).view(B, -1, max_q_len)
/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  agg_num_att = self.softmax(agg_num_att_val.view(-1, max_q_len)).

In [ ]:
best_sel_acc = init_acc[1][0]
best_cond_acc = init_acc[1][1]
best_group_acc = init_acc[1][2]
best_order_acc = init_acc[1][3]
best_tot_acc = 0.0

for i in range(300):
    print('Epoch %d @ %s' % (i + 1, datetime.datetime.now()))
    print(' Loss = %s' % epoch_train(model, optimizer, BATCH_SIZE, sql_data, table_data, schemas, TRAIN_ENTRY))
    train_tot_acc, train_bkd_acc = epoch_acc(model, BATCH_SIZE, sql_data, table_data, schemas, TRAIN_ENTRY,
                                             train_flag=True)
    print(' Train acc_qm: %s' % train_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s' \
          % (train_bkd_acc[0], train_bkd_acc[1], train_bkd_acc[2], train_bkd_acc[3]))

    val_tot_acc, val_bkd_acc = epoch_acc(model, BATCH_SIZE, val_sql_data, val_table_data, schemas, TRAIN_ENTRY,
                                         error_print=False,
                                         train_flag=False)  # for detailed error analysis, pass True to error_print
    print(' Dev acc_qm: %s' % val_tot_acc)
    print(' Breakdown results: sel: %s, cond: %s, group: %s, order: %s' \
          % (val_bkd_acc[0], val_bkd_acc[1], val_bkd_acc[2], val_bkd_acc[3]))

    # save models
    if val_bkd_acc[0] > best_sel_acc:
        best_sel_acc = val_bkd_acc[0]
        print("Saving sel model...")
        torch.save(model.sel_pred.state_dict(), "saved_models/sel_models.dump")
    if val_bkd_acc[1] > best_cond_acc:
        best_cond_acc = val_bkd_acc[1]
        print("Saving cond model...")
        torch.save(model.cond_pred.state_dict(), "saved_models/cond_models.dump")
    if val_bkd_acc[2] > best_group_acc:
        best_group_acc = val_bkd_acc[2]
        print("Saving group model...")
        torch.save(model.group_pred.state_dict(), "saved_models/group_models.dump")
    if val_bkd_acc[3] > best_order_acc:
        best_order_acc = val_bkd_acc[3]
        print("Saving order model...")
        torch.save(model.order_pred.state_dict(), "saved_models/order_models.dump")
    if val_tot_acc > best_tot_acc:
        best_tot_acc = val_tot_acc

    print(' Best val sel = %s, cond = %s, group = %s, order = %s, tot = %s' % (
    best_sel_acc, best_cond_acc, best_group_acc, best_order_acc, best_tot_acc))

Epoch 1 @ 2022-07-02 04:30:45.529831


/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:66: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_prob_qc_num = self.softmax(att_val_qc_num.view((-1, max_q_len))).view(B, -1, max_q_len)
/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:77: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  att_prob_qc = self.softmax(att_val_qc.view((-1, max_q_len))).view(B, -1, max_q_len)
/content/drive/Othercomputers/My MacBook Pro/NLIDB/dataset/spider_graph/baselines/sqlnet/scripts/model/modules/sel_predict.py:119: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  agg_num_att = self.softmax(agg_num_att_val.view(-1, max_q_len)).

 Loss = 6.481789159502302
 Train acc_qm: 0.018571428571428572
 Breakdown results: sel: 0.11242857142857143, cond: 0.4972857142857143, group: 0.6588571428571428, order: 0.7934285714285715
 Dev acc_qm: 0.013539651837524178
 Breakdown results: sel: 0.13056092843326886, cond: 0.5193423597678917, group: 0.6508704061895552, order: 0.793036750483559
Saving sel model...


FileNotFoundError: ignored